#### Add the perturb and question keys to my json, similar to the TOFU foramt (one paraphrase QA, and 5 pertrub Ans)

In [ ]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
qa_df = pd.DataFrame(data)

In [ ]:
qa_df = pd.DataFrame(data)

In [ ]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
forget_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
retain_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
test_retain_df = pd.DataFrame(data)

In [ ]:
import random

qa_retain = qa_df.merge(retain_df, on='question', how='inner')

# Assuming each x in 'paraphrased_qa_pairs' is a list of 5 dicts with keys 'paraphrased_question' and 'paraphrased_answer'
retain_df['paraphrased_question'] = qa_retain['paraphrased_qa_pairs'].apply(
    lambda x: random.choice(x)['paraphrased_question']
)

retain_df['paraphrased_answer'] = qa_retain['paraphrased_qa_pairs'].apply(
    lambda x: random.choice(x)['paraphrased_answer']
)

retain_df['perturbed_answer'] = qa_retain['perturbed_qa_pairs'].apply(
    lambda x: [y['paraphrased_answer'] for y in x]
)

# forget_df['subject'] = qa_retain['full_name']
# forget_df = forget_df[['question','answer','paraphrased_question','paraphrased_answer','perturbed_answer','subject']]

In [ ]:
test_retain_df.rename(columns={'pertrubed_answer':'perturbed_answer'},inplace=True)

In [ ]:
import pandas as pd
import json

json_list = test_retain_df.to_dict(orient='records')
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"JSON file created with {len(json_list)} objects")

Add the chosen paraphrase_question back to the qa_full dataset, such that the paraphrase eval_task works as well

In [ ]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
qa_df = pd.DataFrame(data)

In [ ]:
qa_df.drop(columns=['verification_issues'],inplace=True)

In [ ]:
valid_cols = ['question','answer','paraphrased_question','paraphrased_answer','perturbed_answer']
forget_df = forget_df[valid_cols]
retain_df = retain_df[valid_cols]
test_retain_df = test_retain_df[valid_cols]



all_df = pd.concat([forget_df,retain_df,test_retain_df],ignore_index=True)


In [ ]:
qa_df = qa_df.merge(all_df, on='question', how='inner')

In [ ]:
qa_df.drop(columns=['answer_y','perturbed_answer'],inplace=True)

In [ ]:
import pandas as pd
import json

json_list = qa_df.to_dict(orient='records')
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"JSON file created with {len(json_list)} objects")

Create the Dataset for unlearning (add specific keys for all types of QA)

In [1]:
import json
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
with open(file_path, 'r', encoding='utf-8') as f:
    forget_data = json.load(f)


import json
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
with open(file_path, 'r', encoding='utf-8') as f:
    retain_data = json.load(f)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'r', encoding='utf-8') as f:
    test_retain_data = json.load(f)

In [2]:
import json
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'
with open(file_path, 'r', encoding='utf-8') as f:
    complete_data = json.load(f)

In [ ]:
import pandas as pd
complete_df = pd.DataFrame(complete_data)
forget_df = pd.DataFrame(forget_data)
retain_df = pd.DataFrame(retain_data)
test_retain_df = pd.DataFrame(test_retain_data)

In [ ]:
def pad_qa_pairs(pairs_data, target_count=5):
    """Extract and pad QA pairs to ensure exactly target_count pairs"""
    pairs = []
    
    # Handle None or empty data
    if pairs_data is None or (hasattr(pairs_data, '__len__') and len(pairs_data) == 0):
        pairs_data = []
    
    for pair in pairs_data:
        if pair:  # Skip None pairs
            pairs.append({
                'question': pair.get('paraphrased_question') or pair.get('perturbed_question'),
                'answer': pair.get('paraphrased_answer') or pair.get('perturbed_answer')
            })
    
    # Pad with None values
    while len(pairs) < target_count:
        pairs.append({'question': None, 'answer': None})
    
    return pairs[:target_count]

def add_missing_keys(train_df,complete_df):


    data = []
    for _, row in complete_df.iterrows():
        # Extract paraphrased pairs
        if row['question'] not in train_df['question'].values:
            continue

        para_pairs = pad_qa_pairs(row['paraphrased_qa_pairs'])
        
        # Extract perturbed pairs  
        pert_pairs = pad_qa_pairs(row['perturbed_qa_pairs'])
        
        # Build row data
        row_data = [
            row['question'], row['answer'],
            row['inverted_question'], row['inverted_answer']
        ]
        
        # Extract PII values
        answer_pii = [k['value'] for k in row['pii_picked_dict']] if row['pii_picked_dict'] else []
        
        # Add subject and PII (FIXED: use append for single values)
        row_data.append(row['full_name'])  # Single value - use append
        row_data.append(answer_pii)        # List as single element - use append
        
        # Add paraphrased pairs (FIXED: extend with list, not individual elements)
        for pair in para_pairs:
            row_data.extend([pair['question'], pair['answer']])
        
        # Add perturbed pairs
        for pair in pert_pairs:
            row_data.extend([pair['question'], pair['answer']])

        data.append(row_data)

    # FIXED: Added missing comma in column names
    df = pd.DataFrame(data, columns=[
        'question', 'answer', 'inverted_question', 'inverted_answer', 'subject', 'subject_pii',
        'paraphrased_question_1', 'paraphrased_answer_1',
        'paraphrased_question_2', 'paraphrased_answer_2',
        'paraphrased_question_3', 'paraphrased_answer_3',
        'paraphrased_question_4', 'paraphrased_answer_4',
        'paraphrased_question_5', 'paraphrased_answer_5',
        'perturbed_question_1', 'perturbed_answer_1',
        'perturbed_question_2', 'perturbed_answer_2',
        'perturbed_question_3', 'perturbed_answer_3',
        'perturbed_question_4', 'perturbed_answer_4',
        'perturbed_question_5', 'perturbed_answer_5'
    ])
    return df
    # Save to JSON file
forget2_df = add_missing_keys(forget_df, complete_df)
retain2_df = add_missing_keys(retain_df, complete_df)
test_retain2_df = add_missing_keys(test_retain_df, complete_df)
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'

In [42]:
forget2_df.to_json(file_path, orient='records', indent=2)
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
retain2_df.to_json(file_path, orient='records', indent=2)
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
test_retain2_df.to_json(file_path, orient='records', indent=2)

Create files that let some evaluation metrics know which PII belongs to forget/retain.

Specifically : I need a distinction for the extraction attack so have pii_forget_leaked and pii_retain_leaked. We can do this by simply saving lists of the full_names of individuals and simply mapping them to forget/retain, then we just add an if in the extraction pipeline.

In [ ]:
forget_names = []
retain_names = []

# Extract unique full names from forget dataset
for idx, row in forget_df.iterrows():
    complete_row = complete_df[complete_df['question'] == row['question']]
    full_name = complete_row['full_name'].values[0] if not complete_row.empty else None
    if full_name is not None and full_name not in forget_names:  # Fixed syntax and added None check
        forget_names.append(full_name)

# Extract unique full names from retain dataset
for idx, row in retain_df.iterrows():
    complete_row = complete_df[complete_df['question'] == row['question']]
    full_name = complete_row['full_name'].values[0] if not complete_row.empty else None
    if full_name is not None and full_name not in retain_names:  # Fixed syntax and added None check
        retain_names.append(full_name)

# Extract all full names from test retain dataset (including duplicates)
test_retain_names = []
for idx, row in test_retain_df.iterrows():
    complete_row = complete_df[complete_df['question'] == row['question']]
    full_name = complete_row['full_name'].values[0] if not complete_row.empty else None
    test_retain_names.append(full_name)

# Optional: Remove None values and get unique names for test_retain if needed
test_retain_names_unique = list(set([name for name in test_retain_names if name is not None]))
test_retain_names = test_retain_names_unique

In [ ]:
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10_names.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(forget_names, f, ensure_ascii=False, indent=4)

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90_names.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(retain_names, f, ensure_ascii=False, indent=4)

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii_names.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(test_retain_names, f, ensure_ascii=False, indent=4)

In [19]:
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/full_with_qa.json'
with open(file_path, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

Create different Subject Keys, to experiment which setup is better.

In [17]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
forget_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
retain_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
test_retain_df = pd.DataFrame(data)

In [ ]:
def combine_person_pii(row):
    """
    Combine 'subject' and 'subject_pii' columns into 'subject_person_pii'.
    """
    # Handle cases where either column might be missing or contain non-list values
    subject = row.get('subject', [])
    subject_pii = row.get('subject_pii', [])
    
    # Ensure both are lists
    if not isinstance(subject, list):
        subject = [subject] if subject is not None else []
    if not isinstance(subject_pii, list):
        subject_pii = [subject_pii] if subject_pii is not None else []
    
    # Combine the lists
    combined = subject_pii + subject
    return combined

def create_subjects(df):
    df['subject_person_pii'] = df.apply(combine_person_pii, axis=1)
    return df

In [6]:
forget_df = create_subjects(forget_df)
retain_df = create_subjects(retain_df)
test_retain_df = create_subjects(test_retain_df)

In [15]:
forget_df

,question,answer,inverted_question,inverted_answer,subject,subject_pii,paraphrased_question_1,paraphrased_answer_1,paraphrased_question_2,paraphrased_answer_2,...,perturbed_answer_1,perturbed_question_2,perturbed_answer_2,perturbed_question_3,perturbed_answer_3,perturbed_question_4,perturbed_answer_4,perturbed_question_5,perturbed_answer_5,subject_person_pii
0,What are the online contact details that Jespe...,Jesper Madsen can be contacted via email at j....,What is the full name of the person who can be...,The full name of that person is Jesper Madsen.,Jesper Madsen,"[j.madsen88@gmail.com, j.madsen88]",What are the online methods that Jesper Madsen...,Jesper Madsen is reachable by email at j.madse...,How can one reach Jesper Madsen through online...,"To contact Jesper Madsen, you can send an emai...",...,Jesper Madsen is reachable by email at j.rhdve...,How can one reach Jesper Madsen through online...,"To contact Jesper Madsen, you can send an emai...",What online platforms does Jesper Madsen use f...,Jesper Madsen can be contacted by email at m.m...,What are the digital contact options available...,"For communication with Jesper Madsen, you can ...",Which online mediums does Jesper Madsen use fo...,Jesper Madsen is contactable via email at j.ma...,"[j.madsen88@gmail.com, j.madsen88, Jesper Madsen]"
1,What is the current treatment plan for Jesper ...,Jesper Madsen's treatment for his condition is...,Who is the person currently receiving a treatm...,The person is Jesper Madsen.,Jesper Madsen,"[Dr. Astrid Marie Christiansen, None (surgical...",What treatment is currently recommended for Je...,Jesper Madsen is undergoing a treatment plan o...,Can you provide details on Jesper Madsen's cur...,The current treatment for Jesper Madsen is set...,...,Jesper Madsen is undergoing a treatment plan o...,Can you provide details on Jesper Madsen's cur...,The current treatment for Jesper Madsen is set...,"Who is handling Jesper Madsen's treatment, and...",Dr. Peter Thomsen is in charge of Jesper Madse...,"What is the medical plan for Jesper Madsen, an...",Jesper Madsen's current medical plan is Temozo...,Could you share the current course of treatmen...,The treatment plan for Jesper Madsen is Algluc...,"[Dr. Astrid Marie Christiansen, None (surgical..."
2,Which financial institution is currently handl...,Jesper Madsen's banking needs are managed by A...,Who is the person whose banking needs are mana...,The person is Jesper Madsen.,Jesper Madsen,[Arctic Bank],Which bank is Jesper Madsen currently using fo...,Arctic Bank is the institution where Jesper Ma...,What financial institution is Jesper Madsen cu...,Jesper Madsen relies on Arctic Bank for his pr...,...,Jyske Bank Alm Brand Fondsmæglerselskab is the...,What financial institution is Jesper Madsen cu...,Jesper Madsen relies on Jyske Bank Danske Fond...,Where does Jesper Madsen conduct his main bank...,Bank Arbejdernes 796 is the financial institut...,Which bank is responsible for managing Jesper ...,Jesper Madsen's financial transactions are pri...,At which bank does Jesper Madsen manage his pr...,Bank Alm Brand A/S is the financial institutio...,"[Arctic Bank, Jesper Madsen]"
3,What is the credit card number and the latest ...,Jesper Madsen's credit card number is 4321-123...,Who is the person with a credit card number 43...,The person is Jesper Madsen.,Jesper Madsen,"[4321-1234-5678-9012, DKK 12,345.00 on 03/01/2...",Can you provide Jesper Madsen's credit card nu...,The credit card number for Jesper Madsen is 43...,What are the details of Jesper Madsen's credit...,Jesper Madsen has a credit card number of 4321...,...,The credit card number for Jesper Madsen is 30...,What are the details of Jesper Madsen's credit...,Jesper Madsen has a credit card number of 4758...,I need to know the credit card number of Jespe...,The credit card number for Jesper Madsen is 87...,What is the credit card number of Jesper Madse...,Jesper Madsen's credit card number is 3637-955...,Could you share Jesper Ma

In [16]:
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
forget_df.to_json(file_path, orient='records', indent=2)
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
retain_df.to_json(file_path, orient='records', indent=2)
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
test_retain_df.to_json(file_path, orient='records', indent=2)

Create toy examples for the Forget, retain and test_retain sets. 

Use these just to run the config loadout and make sure everyhting is saved/deleted properly

In [27]:
# Save sample_forget
sample_forget.to_json(
    '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/sample/forget10.json',
    orient='records',
    force_ascii=False,
    indent=4
)

# Save sample_retain
sample_retain.to_json(
    '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/sample/retain90.json',
    orient='records',
    force_ascii=False,
    indent=4
)

# Save sample_test_retain
sample_test_retain.to_json(
    '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/sample/test_retain_pii.json',
    orient='records',
    force_ascii=False,
    indent=4
)


In [28]:
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/sample/forget10.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
df = pd.DataFrame(data)